In [2]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from network import CharacterClassifier
from tqdm import tqdm
from training_data import HASY
from PIL import Image
from matplotlib import pyplot as plt

hasy_train = HASY('HASY')
hasy_test = HASY('HASY', train=False)

print("Train data length: {0}".format(len(hasy_train.data)))
print("Test data length: {0}".format(len(hasy_test.data)))
print("Img Shape: {0}".format(hasy_train.data[0].shape))
print("Number of Labels: {0}".format(hasy_train.no_labels))

100%|██████████| 16992/16992 [00:00<00:00, 103155.04it/s]

Train data length: 1958
Test data length: 224
Img Shape: torch.Size([1, 28, 28])
Number of Labels: 15


In [8]:
from training_data import OwnImgs
from torchvision import models
from torch.nn import Conv2d

# Training Iteration with our images
hasy_model = models.densenet201(num_classes=15)
hasy_model.features.conv0 = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
own_imgs = OwnImgs()

print("Train data length: {0}".format(len(own_imgs.data)))
print("Img Shape: {0}".format(own_imgs.data[0].shape))
print("Number of Labels: {0}".format(own_imgs.no_labels))

img_loader = DataLoader(own_imgs, batch_size=16, shuffle=True)
hasy_model.load_state_dict(torch.load('model_acc_57.58928680419922.ckpt'))
test_loader = DataLoader(hasy_test, batch_size=16, shuffle=False)

# Small accuracy test
accuracies = []
for idx, [x_test, y_test] in enumerate(tqdm(test_loader, desc='Test')):
    test_pred = hasy_model(x_test)
    accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
    accuracies.append(accuracy)
print("Accuracy: {0}".format(np.mean(accuracies)))  

epochs = 15
optimizer = torch.optim.Adam(hasy_model.parameters(), lr=0.001, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(epochs):
    print("Epoch {0}".format(epoch))
    for step, [x_train, y_train] in enumerate(tqdm(img_loader)):
        optimizer.zero_grad()
        train_pred = hasy_model(x_train)
        loss = criterion(train_pred, y_train)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print('Loss: {}'.format(loss))
torch.save(hasy_model.state_dict(), 'hasy_model-trans.ckpt')  

Train data length: 119
Img Shape: torch.Size([1, 28, 28])
Number of Labels: 15


  0%|          | 0/8 [00:00<?, ?it/s]

Accuracy: 25.446428298950195
Epoch 0


 12%|█▎        | 1/8 [00:01<00:12,  1.73s/it]

Loss: 3.389242649078369


...

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 14


 12%|█▎        | 1/8 [00:01<00:12,  1.75s/it]

Loss: 0.22323107719421387


100%|██████████| 8/8 [00:13<00:00,  1.68s/it]
